# CAD2Sees Example: Seismic Assessment Workflow

Complete workflow for seismic assessment of RC frame structures:
1. Parse CAD files → Generate FE model → Run analyses → Assess performance → Visualise results

**Input files**: `3d_v2.dxf`, `sections.dxf`, `general_info.json`

## Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import pickle as pkl

# CAD2Sees modules
from cad2sees.parsing.dxfparse import dxfparse
from cad2sees.model_generation import model_init, node, restraint, geometric_transformation as gt
from cad2sees.model_generation import joint, frame, infill, constraint
from cad2sees.helpers import save_capacity, ops_utils
from cad2sees.analysis import gravity, modal, pushover
from cad2sees.post_processing import demand_capacity, N2, get_failed
from cad2sees.visualise import visualise as vis

## Configuration

In [2]:
# Analysis parameters
LoadPattern = 'Modal'  # 'Modal', 'Uniform', 'Linear', 'Triangular'
Direction = 1          # 1: X-direction, 2: Y-direction

## File Paths

In [3]:
cwd = os.getcwd()

# Input files
dxf_file_3d = os.path.join(cwd, 'inputs', '3D.dxf')
dxf_file_sections = os.path.join(cwd, 'inputs', 'sections.dxf')
json_file_general_info = os.path.join(cwd, 'inputs', 'general_info.json')

# Output directories
output_dir = os.path.join(cwd, 'outputs')
capacity_outdir = os.path.join(output_dir, 'capacity')
modal_outdir = os.path.join(output_dir, 'modal')
push_outdir = os.path.join(output_dir, 'pushover')

# Create directories
for directory in [output_dir, capacity_outdir, modal_outdir, push_outdir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

## 1. Parse CAD Files

In [4]:
# Parse DXF and JSON files
Parser = dxfparse(
    filepath_3D=dxf_file_3d,
    filepath_Sections=dxf_file_sections,
    filepath_GeneralInformation=json_file_general_info
)
Parser.Parse()

Starting DXF parsing workflow...
Step 1: Reorganizing points...
Step 2: Reorganizing elements...
Step 3: Processing sections...
Step 4: Reorganizing infills...
Step 5: Adding mass and loads...
Total load applied: 2566.08
DXF parsing completed successfully!


In [5]:
# Initialize 3D model
_, ndf = model_init.do3D()

## 2. Create Model Components

In [6]:
# Create geometric transformations
Parser.Frames, GeometricTransformations = gt.do(Parser.Frames, Parser.Sections)

In [7]:
# Create nodes and restraints
BCJMap = (np.asarray(Parser.Points['Type']) == 'BCJ')
node.node(IDs=Parser.Points['ID'][~BCJMap], 
          Coordinates=Parser.Points['Coordinates'][~BCJMap])

FixedNodesMap = (Parser.Points['BoundryConditions'].sum(axis=1) == ndf)
restraint.fixall(Parser.Points['ID'][FixedNodesMap])

## 3. Create Elements

In [8]:
# Create beam-column joints
JointOuts = joint.create(Parser.Points, Parser.Frames, Parser.Sections)

In [9]:
# Create frame elements
FrameOuts, SectionOuts = frame.create('BWH_FiberOPS2', Parser.Frames, 
                                      Parser.Sections, Parser.Points, 
                                      GeometricTransformations)

Section ID: Column | Load: [103.68] kN
Section ID: Column | Load: [207.36] kN
Section ID: Column | Load: [224.64] kN
Section ID: Column | Load: [432.] kN
Section ID: Column | Load: [120.96] kN
Section ID: Column | Load: [51.84] kN
Section ID: Column | Load: [112.32] kN
Section ID: Column | Load: [216.] kN
Section ID: Column | Load: [60.48] kN


In [10]:
# Create infill elements
InfillOuts = infill.create(Parser.Infills, Parser.InfillProperties)

## 4. Save Capacities

In [11]:
# Save element capacities
save_capacity.BCJoint(JointOuts, capacity_outdir)
save_capacity.Frame(FrameOuts, capacity_outdir)
save_capacity.Infill(InfillOuts, capacity_outdir)

# Save model data
model_data = {'sections.pkl': SectionOuts, 'frames.pkl': Parser.Frames, 'nodes.pkl': Parser.Points}
for filename, data in model_data.items():
    with open(os.path.join(output_dir, filename), 'wb') as f:
        pkl.dump(data, f)

Beam-Column Joints Data saved to c:\Users\besim\Desktop\CAD2Sees\examples\outputs\capacity\BCJoints.json
Frames Data saved to c:\Users\besim\Desktop\CAD2Sees\examples\outputs\capacity\Frames.json
Infills Data saved to c:\Users\besim\Desktop\CAD2Sees\examples\outputs\capacity\Infills.json


## 5. Apply Constraints

In [12]:
# Define rigid diaphragms
rigid_diaphragm_flag = Parser.config['rigid_diaphragm']
push_nodes = constraint.rigid_diaphragm(Parser.Points, rigid_diaphragm_flag)

In [13]:
# Save model geometry
_ = ops_utils.GetShape(output_dir)

## 6. Run Analyses

In [14]:
# Gravity analysis
gravity.do(Parser.Points)

# Modal analysis
periods, mass_participation_ratios = modal.do(3, outsavemode=1, outdir=modal_outdir, outMPR=True)


Gravity Analysis...
Gravity Analysis Completed!
Calculated Base Reaction: 2566.0799999999995,

Loaded Total Force: 2566.0800000000004,



Using DomainModalProperties - Developed by: Massimo Petracca, Guido Camata, ASDEA Software Technology


In [15]:
# Setup recorders
beam_column_elements = [int(i) for i in list(FrameOuts.keys())]
bcj_elements = [int(i) for i in list(JointOuts.keys())]
infill_elements = [int(i) for i in list(InfillOuts.keys())]

ops_utils.AddDisplacementRecorders(push_outdir)
ops_utils.AddChordRotationRecorders(push_outdir, beam_column_elements)
ops_utils.AddFrameForceRecorders(push_outdir, beam_column_elements)
ops_utils.AddFrameInflectionPointRecorders(push_outdir, beam_column_elements)
ops_utils.AddBCJRecorder(push_outdir, bcj_elements)
ops_utils.AddInfillRecorder(push_outdir, infill_elements)

In [16]:
# Pushover analysis
spo_curve, spo_info = pushover.do(LoadPattern, Direction, push_nodes, 
                                  nsStep=1000, dref=0.1, mu=3.0)

# Save results
np.savetxt(os.path.join(push_outdir, 'SPOCurve.out'), spo_curve)
np.savetxt(os.path.join(push_outdir, 'PushInfo.out'), spo_info)

Modal pushover for Mode: 1 is being defined...
Direction 2


WARNING can't set handler after analysis is created


DispControl Analysis SUCCESSFUL


## 7. Post-Processing

In [17]:
# Calculate demand-capacity ratios
demand_capacity.BCJ(push_outdir, capacity_outdir)
demand_capacity.FrameRotation(push_outdir, capacity_outdir)
demand_capacity.Infill(push_outdir, capacity_outdir)
demand_capacity.FrameShear(push_outdir, capacity_outdir, addInfill=1, priestleyFlag=0)

## 8. N2 Method

In [18]:
# Extract seismic parameters
pga = Parser.config['PGA']
soil_class = Parser.config['SoilClass']
nation = Parser.config['NationalAnnex']
spectrum_type = Parser.config['SpectrumType']

# Apply N2 method
n2_proc = N2.N2(pga, soil_class, spo_curve, spo_info, push_outdir,
                capacity_outdir, Nation=nation, SpectrumType=spectrum_type)
n2_proc.do()
n2_proc.plot_it()

Displacement Demand: 0.165 [m] Being Reached at Step 549


## 9. Failure Assessment

In [19]:
# Get failure data at performance point
frame_rotation_fail, frame_yeild = get_failed.getFrameFlexural(push_outdir, int(n2_proc.StepNum))
frame_shear_fail = get_failed.getFrameShear(push_outdir, int(n2_proc.StepNum))

# Organize DCR data
DFCols = list(frame_rotation_fail.keys())
data_dict = {'YeildI': [], 'YeildJ': [], 'RotationI': [], 'RotationJ': [], 'Rotation': [], 'Shear': []}

for k in DFCols:
    rot_i = frame_rotation_fail[k].get('I', {})
    rot_j = frame_rotation_fail[k].get('J', {})
    rot_vals = [v for d in [rot_i, rot_j] for v in d.values()]
    data_dict['Rotation'].append(max(rot_vals) if rot_vals else float('nan'))
    data_dict['RotationI'].append(max(rot_i.values()) if rot_i else float('nan'))
    data_dict['RotationJ'].append(max(rot_j.values()) if rot_j else float('nan'))

    yield_i = frame_yeild[k].get('I', {})
    yield_j = frame_yeild[k].get('J', {})
    data_dict['YeildI'].append(max(yield_i.values()) if yield_i else float('nan'))
    data_dict['YeildJ'].append(max(yield_j.values()) if yield_j else float('nan'))

    shear_i = frame_shear_fail[k].get('I', {})
    shear_j = frame_shear_fail[k].get('J', {})
    shear_vals = [v for d in [shear_i, shear_j] for v in d.values()]
    data_dict['Shear'].append(max(shear_vals) if shear_vals else float('nan'))

df = pd.DataFrame(data_dict, index=DFCols).T
shear_collapses = df.columns[df.loc['Shear'] >= 1.0]
rotation_collapses = df.columns[df.loc['Rotation'] >= 1.0]

## 10. Visualisation

In [21]:
# Create visualisations
V = vis.visualise(output_dir)

# Modal visualisation
V.modal(1, scalefactor=10, showme=1, saveme='mp4')

# Pushover visualisation
V.Pushover(push_outdir, showDCRframeShear=True, step=int(n2_proc.StepNum), 
           showme=0, saveme=1)

Writing mp4 frames for modal animation...
Animation saved to c:\Users\besim\Desktop\CAD2Sees\examples\outputs\Modal\modal_Mode1.mp4
Number of Cracked Beam Column Joints 5
IDs of Cracked Beam Column Joints [1118. 1119. 1122. 1124. 1125.]
Number of Peak Beam Column Joints 9
IDs of Peak Beam Column Joints [1109. 1110. 1111. 1112. 1113. 1114. 1115. 1116. 1117.]
Number of Ultimate Beam Column Joints 0
IDs of Ultimate Beam Column Joints []
Number of Frames Damage Limitation 8
IDs of Frames Damage Limitation [np.float64(101110.0), np.float64(107116.0), np.float64(102111.0), np.float64(105114.0), np.float64(108117.0), np.float64(100109.0), np.float64(103112.0), np.float64(106115.0)]
Number of Frames Significant Damage 0
IDs of Frames Significant Damage []
Number of Frames Near Collapse 1
IDs of Frames Near Collapse [np.float64(104113.0)]
Pushover plot saved to c:\Users\besim\Desktop\CAD2Sees\examples\outputs\pushover\PushoverResultsPlot.html


In [23]:
# Results summary
print(f"Shear failures: {len(shear_collapses)}")
print(f"Rotation failures: {len(rotation_collapses)}")

Shear failures: 0
Rotation failures: 1


## ✅ Analysis Complete

Results saved to `outputs/` directory:
- Capacity curves and DCR assessments
- Modal analysis results  
- Pushover curves and performance point
- Interactive 3D visualisations